In [1]:
import os

In [2]:
from sklearn.metrics import confusion_matrix
from dice_loss import IoULoss

In [3]:
pred = [True,False,True,False,True,False,True,False]

In [4]:
true = [True,True,False,True,False,True,True,False]

In [5]:
confusion_matrix(1*true,1*pred)

array([[1, 2],
       [3, 2]])

In [6]:
iou = IoULoss() 

In [7]:
iou(pred, true)

AttributeError: 'list' object has no attribute 'shape'

In [8]:
import numpy as np

In [9]:
1568945/(1568945+477352+491671)

0.618189433436513

In [10]:
import torch

In [11]:
nb_samples = 5
nb_dim = 3
nb_classes = 4
output = torch.randn(nb_samples,nb_classes, nb_dim, nb_dim)
pred = torch.argmax(output, 1)
target = torch.randint(0, 2, (nb_samples,nb_dim,nb_dim))

In [12]:
iou(target,target)

tensor(-1.)

In [22]:
[i == j for i, j in zip(target.shape, pred.shape)]

[True, True, True]

In [25]:
target.sum().item()

25

In [26]:
target.nelement() - target.sum().item()

20

In [20]:
target

tensor([[[0, 1, 0],
         [1, 0, 1],
         [0, 1, 1]],

        [[1, 1, 0],
         [0, 0, 1],
         [0, 1, 1]],

        [[1, 0, 0],
         [1, 0, 0],
         [1, 0, 0]],

        [[1, 1, 1],
         [0, 0, 0],
         [1, 1, 0]],

        [[1, 1, 1],
         [1, 1, 1],
         [0, 1, 0]]])

In [19]:
iou(pred,target)

TypeError: forward() got an unexpected keyword argument 'mask'

In [15]:
iou(target,pred)

tensor(-0.8514)

In [16]:
iou(target,torch.zeros_like(target))

tensor(-0.4278)

In [17]:
iou(target,torch.ones_like(target))

tensor(-0.6667)

In [10]:
num_channels=2

In [11]:
target = torch.randint(0, 2, (nb_samples,num_channels,nb_dim,nb_dim))

In [12]:
target.shape

torch.Size([5, 2, 3, 3])

In [13]:
target.nelement()

90

In [14]:
target.sum().item()

36

In [16]:
torch.save({'target':target, 'pred': pred}, 'dummy_dict.dict')

In [14]:
weight_ = torch.zeros_like(target)

In [17]:
device = "cpu"

In [22]:
def create_conf_matrix2(target, pred, debug=True):
    import sys
    
    target = target.reshape(-1).cpu().numpy()
    pred = pred.reshape(-1).cpu().numpy()
    
        
    conf_matrix = torch.from_numpy(confusion_matrix(target, pred)).to(device)
    
    print('Threat Score: {}'.format((1.0*conf_matrix[1,1])/(conf_matrix[1,1]+conf_matrix[1,0]+conf_matrix[0,1])))
    
    return conf_matrix

def classScores(conf_matrix):
    print('Confusion matrix\n', conf_matrix)
    TP = conf_matrix.diag()
    TN = torch.zeros_like(TP)
    FP = torch.zeros_like(TP)
    FN = torch.zeros_like(TP)
    for c in range(conf_matrix.size(0)):
        idx = torch.ones(conf_matrix.size(0)).byte()
        idx[c] = 0
        # all non-class samples classified as non-class
        TN[c] = conf_matrix[idx.nonzero()[:, None], idx.nonzero()].sum() #conf_matrix[idx[:, None], idx].sum() - conf_matrix[idx, c].sum()
        # all non-class samples classified as class
        FP[c] = conf_matrix[idx, c].sum()
        # all class samples not classified as class
        FN[c] = conf_matrix[c, idx].sum()

        print('Class {}\nTP {}, TN {}, FP {}, FN {}'.format(
            c, TP[c], TN[c], FP[c], FN[c]))
        
    return (TP.detach().cpu().numpy(), 
            TN.detach().cpu().numpy(), 
            FP.detach().cpu().numpy(), 
            FN.detach().cpu().numpy())

In [23]:
classScores(create_conf_matrix2(target, target))

Threat Score: 1.0
Confusion matrix
 tensor([[45,  0],
        [ 0, 45]])
Class 0
TP 45, TN 45, FP 0, FN 0
Class 1
TP 45, TN 45, FP 0, FN 0


/opt/conda/conda-bld/pytorch_1579022027550/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1579022027550/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1579022027550/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1579022027550/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


(array([45, 45]), array([45, 45]), array([0, 0]), array([0, 0]))

In [16]:
target[:,channel,:,:].data.shape

NameError: name 'channel' is not defined

In [24]:
cur_target = target[:,0,:,:]

In [25]:
cur_target.shape

torch.Size([5, 3, 3])

In [26]:
cur_target.data.view(-1).long()

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [32]:
target[:,:,:,:].data.reshape(-1).long().shape

torch.Size([90])

In [ ]:
weight[target[:,0,:,:].data.reshape(-1).long()]

In [22]:
weights = [[0.1, 0.9],[0.01, 0.99]]

In [39]:
weight_ = torch.zeros_like(target, dtype=torch.float)
for channel in range(num_channels):
    cur_target = target[:,channel,:,:]
    weight = torch.tensor(weights[channel])
    weight_[:,channel,:,:] += weight[cur_target.data.reshape(-1).long()].view_as(cur_target)

In [44]:
target.size(1)

2

In [40]:
weight_.shape

torch.Size([5, 2, 3, 3])

In [48]:
target.contiguous()

torch.Size([5, 2, 3, 3])

In [53]:
(target.contiguous() * target.contiguous())

torch.Size([5, 2, 3])

In [55]:
intersection = (target.contiguous() * target.contiguous()).sum(dim=2).sum(dim=2)

In [56]:
smooth = 1.0

In [59]:
(1 - ((2. * intersection + smooth) / (target.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth))).mean()

tensor(0.)

In [63]:
pred = torch.randint(0, 2, (nb_samples,num_channels,nb_dim,nb_dim))

In [64]:
pred.shape

torch.Size([5, 2, 3, 3])

In [65]:
intersection = (pred * target).sum(dim=2).sum(dim=2)
    
(1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))

tensor([[0.8889, 0.4444],
        [0.4444, 0.1818],
        [0.5000, 0.2667],
        [0.5455, 0.3750],
        [0.0833, 0.3636]])

In [43]:
weight_[:,1,:,:]

tensor([[[0.9900, 0.0100, 0.0100],
         [0.0100, 0.9900, 0.0100],
         [0.0100, 0.9900, 0.9900]],

        [[0.0100, 0.0100, 0.9900],
         [0.0100, 0.0100, 0.9900],
         [0.9900, 0.9900, 0.0100]],

        [[0.9900, 0.9900, 0.9900],
         [0.9900, 0.0100, 0.9900],
         [0.0100, 0.9900, 0.9900]],

        [[0.0100, 0.9900, 0.9900],
         [0.0100, 0.9900, 0.9900],
         [0.0100, 0.0100, 0.0100]],

        [[0.9900, 0.0100, 0.0100],
         [0.9900, 0.0100, 0.9900],
         [0.9900, 0.9900, 0.0100]]])

In [31]:
weight_[:,1,:,:]

tensor([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]])

In [17]:
weight = torch.tensor([0.1, 0.9])
weight_ = weight[target.data.view(-1).long()].view_as(target)

In [19]:
weight_

tensor([[[0.9000, 0.1000, 0.9000],
         [0.1000, 0.9000, 0.9000],
         [0.9000, 0.1000, 0.9000]],

        [[0.1000, 0.1000, 0.9000],
         [0.1000, 0.1000, 0.9000],
         [0.9000, 0.9000, 0.1000]],

        [[0.9000, 0.1000, 0.1000],
         [0.9000, 0.9000, 0.1000],
         [0.9000, 0.1000, 0.9000]],

        [[0.1000, 0.9000, 0.1000],
         [0.9000, 0.9000, 0.9000],
         [0.1000, 0.1000, 0.1000]],

        [[0.9000, 0.9000, 0.1000],
         [0.9000, 0.9000, 0.1000],
         [0.1000, 0.1000, 0.1000]]])

In [99]:
pred.shape

torch.Size([5, 3, 3])

In [100]:
target.shape

torch.Size([5, 3, 3])

In [101]:
target

tensor([[[2, 1, 2],
         [0, 1, 1],
         [0, 2, 1]],

        [[1, 0, 1],
         [2, 3, 2],
         [0, 2, 0]],

        [[2, 2, 2],
         [0, 0, 0],
         [1, 1, 1]],

        [[0, 1, 3],
         [0, 2, 0],
         [0, 2, 3]],

        [[1, 3, 1],
         [0, 0, 1],
         [1, 3, 2]]])

In [102]:
target = torch.randint(0, 2, (nb_samples,nb_dim,nb_dim,3))

In [103]:
values, indices = torch.max(target,1)

In [104]:
y_target = torch.zeros((nb_samples,nb_dim,nb_dim), dtype=torch.long)

In [105]:
y_target

tensor([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]])

In [106]:
y_target[values > 0] = indices[values > 0]

In [107]:
y_target

tensor([[[1, 2, 2],
         [0, 1, 2],
         [2, 1, 2]],

        [[2, 2, 2],
         [1, 0, 0],
         [2, 2, 0]],

        [[2, 1, 0],
         [2, 2, 2],
         [1, 1, 0]],

        [[0, 0, 0],
         [1, 2, 2],
         [1, 0, 2]],

        [[0, 2, 0],
         [0, 0, 0],
         [2, 2, 0]]])

In [108]:
target

tensor([[[[0, 1, 1],
          [1, 0, 0],
          [0, 1, 1]],

         [[1, 0, 0],
          [0, 1, 0],
          [0, 1, 0]],

         [[0, 1, 1],
          [0, 0, 1],
          [1, 0, 1]]],


        [[[0, 1, 0],
          [0, 0, 1],
          [1, 0, 0]],

         [[1, 1, 0],
          [1, 0, 0],
          [1, 1, 0]],

         [[1, 1, 1],
          [0, 0, 0],
          [1, 1, 0]]],


        [[[0, 1, 1],
          [0, 0, 0],
          [0, 1, 1]],

         [[0, 1, 0],
          [0, 1, 1],
          [1, 1, 0]],

         [[1, 0, 0],
          [1, 1, 1],
          [0, 0, 0]]],


        [[[1, 0, 1],
          [0, 1, 1],
          [1, 0, 1]],

         [[0, 0, 0],
          [1, 1, 1],
          [1, 0, 1]],

         [[0, 0, 0],
          [0, 1, 1],
          [0, 0, 1]]],


        [[[1, 1, 0],
          [0, 0, 0],
          [0, 1, 1]],

         [[0, 0, 0],
          [0, 0, 0],
          [1, 0, 0]],

         [[0, 1, 0],
          [0, 0, 0],
          [1, 1, 0]]]])

In [109]:
target.reshape(-1)

tensor([0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
        1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1,
        0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
        1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1,
        1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0])

In [110]:
pred

tensor([[[0, 2, 2],
         [1, 2, 2],
         [3, 1, 3]],

        [[1, 2, 0],
         [2, 2, 2],
         [0, 1, 3]],

        [[1, 2, 3],
         [3, 0, 2],
         [3, 2, 3]],

        [[3, 2, 2],
         [0, 3, 1],
         [2, 3, 2]],

        [[3, 2, 2],
         [1, 2, 3],
         [0, 2, 2]]])

In [111]:
pred[0].reshape(-1)

tensor([0, 2, 2, 1, 2, 2, 3, 1, 3])

In [112]:
len(target.unique())

2

In [114]:
max(pred.unique())

tensor(3)

In [94]:
def create_conf_matrix(target, pred):
    import sys
    
    target = target.reshape(-1)
    pred = pred.reshape(-1)
    
    nb_classes = max(target.unique())
    if len(pred.unique()) > nb_classes:
        print('More predicted classes than true classes')
        sys.exit(1)
        
    conf_matrix = torch.zeros(nb_classes, nb_classes)
    for t, p in zip(target, pred):
        conf_matrix[t, p] += 1
    
    return conf_matrix

In [95]:
create_conf_matrix(target,pred)

tensor([[12., 16.],
        [ 9.,  8.]])

In [88]:
%tb

SystemExit: 1

In [71]:
conf_matrix = torch.zeros(nb_classes, nb_classes)
for b in range(nb_samples): 
    for t, p in zip(target[b], pred[b]):
        print('t',t)
        print('p',p)
        conf_matrix[t, p] += 1
        print(conf_matrix[t, p])
    print('-'*50)

print('Confusion matrix\n', conf_matrix)

t tensor([1, 1, 0])
p tensor([0, 0, 1])
tensor([1., 1., 1.])
t tensor([0, 0, 1])
p tensor([1, 1, 1])
tensor([2., 2., 1.])
t tensor([1, 0, 0])
p tensor([0, 1, 0])
tensor([2., 3., 1.])
--------------------------------------------------
t tensor([0, 1, 1])
p tensor([1, 0, 0])
tensor([4., 3., 3.])
t tensor([1, 0, 0])
p tensor([1, 0, 0])
tensor([2., 2., 2.])
t tensor([1, 0, 0])
p tensor([0, 1, 0])
tensor([4., 5., 3.])
--------------------------------------------------
t tensor([0, 0, 1])
p tensor([1, 0, 0])
tensor([6., 4., 5.])
t tensor([0, 0, 0])
p tensor([0, 0, 1])
tensor([5., 5., 7.])
t tensor([0, 0, 1])
p tensor([0, 1, 0])
tensor([6., 8., 6.])
--------------------------------------------------
t tensor([0, 0, 0])
p tensor([0, 1, 1])
tensor([7., 9., 9.])
t tensor([0, 0, 1])
p tensor([0, 1, 1])
tensor([ 8., 10.,  3.])
t tensor([0, 0, 1])
p tensor([0, 1, 0])
tensor([ 9., 11.,  7.])
--------------------------------------------------
t tensor([0, 1, 1])
p tensor([1, 1, 1])
tensor([12.,  4., 

In [31]:
def classScores(conf_matrix):
    TP = conf_matrix.diag()
    TN = torch.zeros_like(TP)
    FP = torch.zeros_like(TP)
    FN = torch.zeros_like(TP)
    for c in range(conf_matrix.size(0)):
        idx = torch.ones(conf_matrix.size(0)).byte()
        idx[c] = 0
        # all non-class samples classified as non-class
        TN[c] = conf_matrix[idx.nonzero()[:, None], idx.nonzero()].sum() #conf_matrix[idx[:, None], idx].sum() - conf_matrix[idx, c].sum()
        # all non-class samples classified as class
        FP[c] = conf_matrix[idx, c].sum()
        # all class samples not classified as class
        FN[c] = conf_matrix[c, idx].sum()

        print('Class {}\nTP {}, TN {}, FP {}, FN {}'.format(
            c, TP[c], TN[c], FP[c], FN[c]))
        
    return TP, TN, FP, FN

In [32]:
print('TP {}, TN {}, FP {}, FN {}'.format(*classScores(conf_matrix)))

Class 0
TP 380.0, TN 2800.0, FP 1000.0, FN 980.0
Class 1
TP 340.0, TN 2980.0, FP 880.0, FN 960.0
Class 2
TP 280.0, TN 2980.0, FP 920.0, FN 980.0
Class 3
TP 340.0, TN 2900.0, FP 1020.0, FN 900.0
TP tensor([380., 340., 280., 340.]), TN tensor([2800., 2980., 2980., 2900.]), FP tensor([1000.,  880.,  920., 1020.]), FN tensor([980., 960., 980., 900.])


/opt/conda/conda-bld/pytorch_1579022027550/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1579022027550/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1579022027550/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1579022027550/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1579022027550/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_15790220275

In [5]:
target

tensor([3, 1, 2, 2, 2, 1, 3, 1, 2, 2, 0, 3, 3, 3, 2, 0, 1, 2, 0, 2])

In [11]:
TP

tensor([0., 0., 1., 2.])

In [12]:
TN

tensor([11., 14.,  8., 10.])

In [13]:
FP

tensor([7., 1., 5., 4.])

In [14]:
FN

tensor([2., 5., 6., 4.])